# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,53.20,88,70,10.58,RU,Wed Sep 11 21:53:35 2024
1,1,port-aux-francais,-49.3500,70.2167,38.50,86,100,41.14,TF,Wed Sep 11 21:53:35 2024
2,2,thompson,55.7435,-97.8558,55.56,62,20,2.30,CA,Wed Sep 11 21:52:43 2024
3,3,ilulissat,69.2167,-51.1000,37.42,70,87,1.14,GL,Wed Sep 11 21:51:59 2024
4,4,merizo village,13.2658,144.6691,83.98,90,100,25.10,GU,Wed Sep 11 21:53:36 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
# Display the map
city_data_df.hvplot.points('Lng','Lat',color='City',tiles='OSM',size='Humidity',geo=True)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
hotel_df = city_data_df[(city_data_df['Max Temp']>70)&(city_data_df['Max Temp']<90)&(city_data_df.Humidity<20)].dropna()

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,tanumah,27.1000,44.1333,83.66,18,33,5.55,SA,Wed Sep 11 21:53:45 2024
216,216,paulden,34.8856,-112.4682,84.38,19,0,11.01,US,Wed Sep 11 21:54:00 2024
235,235,mount isa,-20.7333,139.5000,89.37,17,0,13.80,AU,Wed Sep 11 21:54:02 2024
292,292,as sulayyil,20.4607,45.5779,84.90,13,11,3.36,SA,Wed Sep 11 21:54:08 2024
337,337,ivins,37.1686,-113.6794,89.96,12,0,1.99,US,Wed Sep 11 21:54:13 2024
388,388,ardakan,32.3100,54.0175,75.56,15,0,4.99,IR,Wed Sep 11 21:54:18 2024
447,447,kazerun,29.6195,51.6541,82.35,19,0,4.43,IR,Wed Sep 11 21:54:25 2024
468,468,riverton,40.5219,-111.9391,84.69,17,100,18.41,US,Wed Sep 11 21:49:55 2024
497,497,al hazm,16.1641,44.7769,82.36,17,92,2.55,YE,Wed Sep 11 21:54:30 2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
89,tanumah,SA,27.1000,44.1333,18,
216,paulden,US,34.8856,-112.4682,19,
235,mount isa,AU,-20.7333,139.5000,17,
292,as sulayyil,SA,20.4607,45.5779,13,
337,ivins,US,37.1686,-113.6794,12,
388,ardakan,IR,32.3100,54.0175,15,
447,kazerun,IR,29.6195,51.6541,19,
468,riverton,US,40.5219,-111.9391,17,
497,al hazm,YE,16.1641,44.7769,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
name_address

{'statusCode': 400,
 'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types'}

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row.Lat
    lng = row.Lng

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tanumah - nearest hotel: No hotel found
paulden - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
as sulayyil - nearest hotel: No hotel found
ivins - nearest hotel: Crescent Moon Inn
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
kazerun - nearest hotel: هتل بیشاپور
riverton - nearest hotel: Homewood Suites
al hazm - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
89,tanumah,SA,27.1000,44.1333,18,No hotel found
216,paulden,US,34.8856,-112.4682,19,No hotel found
235,mount isa,AU,-20.7333,139.5000,17,Ibis Styles
292,as sulayyil,SA,20.4607,45.5779,13,No hotel found
337,ivins,US,37.1686,-113.6794,12,Crescent Moon Inn
388,ardakan,IR,32.3100,54.0175,15,هتل رستوران باغ سرای اردکان
447,kazerun,IR,29.6195,51.6541,19,هتل بیشاپور
468,riverton,US,40.5219,-111.9391,17,Homewood Suites
497,al hazm,YE,16.1641,44.7769,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
89,tanumah,SA,27.1000,44.1333,18,No hotel found
216,paulden,US,34.8856,-112.4682,19,No hotel found
235,mount isa,AU,-20.7333,139.5000,17,Ibis Styles
292,as sulayyil,SA,20.4607,45.5779,13,No hotel found
337,ivins,US,37.1686,-113.6794,12,Crescent Moon Inn
388,ardakan,IR,32.3100,54.0175,15,هتل رستوران باغ سرای اردکان
447,kazerun,IR,29.6195,51.6541,19,هتل بیشاپور
468,riverton,US,40.5219,-111.9391,17,Homewood Suites
497,al hazm,YE,16.1641,44.7769,17,No hotel found


In [47]:
%%capture --no-display

# Configure the map plot
# Display the map
hotel_df.hvplot.points('Lng','Lat',color='City',tiles='OSM',size='Humidity',geo=True, frame_height=500, hover_cols=['Hotel Name','Country'])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)